In [1]:
# Necessary libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_excel('/content/drive/MyDrive/Files/inputfile.xlsx') # Directory to your file
df

,Vocab,Meaning,Synonym,In a sentence
0,Alacrity,NaN,NaN,NaN
1,Aspirant,NaN,NaN,NaN
2,Belittle,NaN,NaN,NaN
3,Belligerent,NaN,NaN,NaN
4,Brash,NaN,NaN,NaN
5,Disdain,NaN,NaN,NaN
6,Dregs,NaN,NaN,NaN
7,Feint,NaN,NaN,NaN
8,Frenzy,NaN,NaN,NaN
9,Intimidate,NaN,NaN,NaN


In [4]:
# Get list of words
words = df['Vocab'].tolist()
words

['Alacrity',
 'Aspirant',
 'Belittle',
 'Belligerent',
 'Brash',
 'Disdain',
 'Dregs',
 'Feint',
 'Frenzy',
 'Intimidate',
 'Laceration',
 'Octogenarian',
 'Promulgate',
 'Pugnacious',
 'Scoff',
 'Scurrilous',
 'Solace',
 'Sordid',
 'Tangible']

In [5]:
# Meaning column
meaning = []

def get_definition(word):
    # Format the word for URL
    formatted_word = word.replace(" ", "-")

    # Create the URL
    url = f"https://www.oxfordlearnersdictionaries.com/definition/english/{formatted_word}_1?q={word}"

    # Headers to mimic a browser request
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }

    # Setting up a session with retries
    session = requests.Session()
    retries = Retry(total=5, backoff_factor=0.1, status_forcelist=[500, 502, 503, 504])
    session.mount('https://', HTTPAdapter(max_retries=retries))

    try:
        # Send a GET request to the URL
        response = session.get(url, headers=headers)

        # Check if the response is successful
        if response.status_code != 200:
            return f"Error: Unable to fetch data for '{word}' (status code: {response.status_code})"

        # Parse the content with BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extract the definition
        definitions = []
        for span in soup.find_all('span', class_='def'):
            definitions.append(span.text.strip())

        if not definitions:
            return f"No definitions found for '{word}'"

        return definitions

    except requests.exceptions.RequestException as e:
        return f"Error: Definittion for {word} not found"

    finally:
        session.close()


for word in words:
  definitions = get_definition(word.lower())
  meaning.append(definitions[0])


In [6]:
df['Meaning'] = meaning
df

,Vocab,Meaning,Synonym,In a sentence
0,Alacrity,great happiness or enthusiasm,NaN,NaN
1,Aspirant,wanting to start the career or activity that i...,NaN,NaN
2,Belittle,"to make somebody, or the things that somebody ...",NaN,NaN
3,Belligerent,aggressive and unfriendly,NaN,NaN
4,Brash,confident in an aggressive way,NaN,NaN
5,Disdain,the feeling that somebody/something is not goo...,NaN,NaN
6,Dregs,"the last drops of a liquid, mixed with little ...",NaN,NaN
7,Feint,(especially in sport) a movement that is inten...,NaN,NaN
8,Frenzy,a state of great activity and strong emotion t...,NaN,NaN
9,Intimidate,to frighten or threaten somebody so that they ...,NaN,NaN


In [7]:
# Synonym column
synonym = []

def get_synonyms(word):
    # Format the word for URL
    formatted_word = word.replace(" ", "-")

    # Create the URL
    url = f"https://www.thesaurus.com/browse/{word}"

    # Headers to mimic a browser request
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }

    # Setting up a session with retries
    session = requests.Session()
    retries = Retry(total=5, backoff_factor=0.1, status_forcelist=[500, 502, 503, 504])
    session.mount('https://', HTTPAdapter(max_retries=retries))

    try:
        # Send a GET request to the URL
        response = session.get(url, headers=headers)

        # Check if the response is successful
        if response.status_code != 200:
            return f"Error: Unable to fetch data for '{word}' (status code: {response.status_code})"

        # Parse the content with BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extract the synonyms
        synonyms = []
        for a in soup.find_all('a', class_='Bf5RRqL5MiAp4gB8wAZa'):
            synonyms.append(a.text.strip())

        if not synonyms:
            return f"No synonyms found for '{word}'"

        return synonyms

    except requests.exceptions.RequestException as e:
        return f"Error: {e}"

    finally:
        session.close()


for word in words:
    synonyms = get_synonyms(word.lower())
    synonym.append(str(synonyms).replace('[','').replace(']','').replace("'",""))

In [8]:
synonym

['No synonyms found for alacrity',
 'applicant, candidate, contestant',
 'criticize, decry, deride, discredit, disparage, downplay, scorn, underestimate',
 'aggressive, antagonistic, bellicose, combative, contentious, hostile, ornery, quarrelsome',
 'bold, brazen, cheeky, cocky, rude, foolhardy, impetuous, overconfident, rash, reckless',
 'antipathy, arrogance, aversion, contempt, derision, dislike, hatred, pride, ridicule, scorn, abhor, belittle, deride, despise, pooh-pooh, spurn',
 'No synonyms found for dregs',
 'ruse',
 'burst, craze, excitement, fever, free-for-all, furor, fury, hysteria, madness, mania, outburst, passion, rage, turmoil',
 'alarm, awe, bully, coerce, constrain, daunt, dishearten, dismay, scare, subdue, terrify, terrorize',
 'lesion',
 'Error: Unable to fetch data for octogenarian (status code: 404)',
 'declare, notify, promote, publish',
 'No synonyms found for pugnacious',
 'belittle, deride, jeer, mock, reject, ridicule, scorn, sneer',
 'defamatory, indecent, in

In [9]:
df['Synonym'] = synonym
df

,Vocab,Meaning,Synonym,In a sentence
0,Alacrity,great happiness or enthusiasm,No synonyms found for alacrity,NaN
1,Aspirant,wanting to start the career or activity that i...,"applicant, candidate, contestant",NaN
2,Belittle,"to make somebody, or the things that somebody ...","criticize, decry, deride, discredit, disparage...",NaN
3,Belligerent,aggressive and unfriendly,"aggressive, antagonistic, bellicose, combative...",NaN
4,Brash,confident in an aggressive way,"bold, brazen, cheeky, cocky, rude, foolhardy, ...",NaN
5,Disdain,the feeling that somebody/something is not goo...,"antipathy, arrogance, aversion, contempt, deri...",NaN
6,Dregs,"the last drops of a liquid, mixed with little ...",No synonyms found for dregs,NaN
7,Feint,(especially in sport) a movement that is inten...,ruse,NaN
8,Frenzy,a state of great activity and strong emotion t...,"burst, craze, excitement, fever, free-for-all,...",NaN
9,Intimidate,to frighten or threaten somebody so that they ...,"alarm, awe, bully, coerce, constrain, daunt, d...",NaN


In [10]:
# Example column
example = []

def get_example(word):
    # Format the word for URL
    formatted_word = word.replace(" ", "-")

    # Create the URL
    url = f"https://www.oxfordlearnersdictionaries.com/definition/english/{formatted_word}_1?q={word}"

    # Headers to mimic a browser request
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }

    # Setting up a session with retries
    session = requests.Session()
    retries = Retry(total=5, backoff_factor=0.1, status_forcelist=[500, 502, 503, 504])
    session.mount('https://', HTTPAdapter(max_retries=retries))

    try:
        # Send a GET request to the URL
        response = session.get(url, headers=headers)

        # Check if the response is successful
        if response.status_code != 200:
            return f"Error: Unable to fetch data for '{word}' (status code: {response.status_code})"

        # Parse the content with BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extract the example
        examples = []
        for span in soup.find_all('span', class_='x'):
            examples.append(span.text.strip())

        if not examples:
            return f"No examples found for '{word}'"

        return examples

    except requests.exceptions.RequestException as e:
        return f"Error: Examples for {word} not found"

    finally:
        session.close()

for word in words:
  examples = get_example(word.lower())
  example.append(examples[0])

In [11]:
example

['They accepted the offer with alacrity.',
 'an aspirant writer',
 'She felt her husband constantly belittled her achievements.',
 'a belligerent attitude',
 "Beneath his brash exterior, he's still a little boy inside.",
 'She treated him with disdain.',
 'coffee dregs',
 'N',
 'in a frenzy of activity/excitement/violence',
 'They were accused of intimidating people into voting for them.',
 'She suffered multiple lacerations to the face.',
 'N',
 'The new constitution was promulgated in 2006.',
 'N',
 "Don't scoff—she's absolutely right.",
 'scurrilous rumours',
 'She smiled, as though solaced by the memory.',
 'It was a shock to discover the truth about his sordid past.',
 'tangible benefits/improvements/results, etc.']

In [12]:
df['In a sentence'] = example
df

,Vocab,Meaning,Synonym,In a sentence
0,Alacrity,great happiness or enthusiasm,No synonyms found for alacrity,They accepted the offer with alacrity.
1,Aspirant,wanting to start the career or activity that i...,"applicant, candidate, contestant",an aspirant writer
2,Belittle,"to make somebody, or the things that somebody ...","criticize, decry, deride, discredit, disparage...",She felt her husband constantly belittled her ...
3,Belligerent,aggressive and unfriendly,"aggressive, antagonistic, bellicose, combative...",a belligerent attitude
4,Brash,confident in an aggressive way,"bold, brazen, cheeky, cocky, rude, foolhardy, ...","Beneath his brash exterior, he's still a littl..."
5,Disdain,the feeling that somebody/something is not goo...,"antipathy, arrogance, aversion, contempt, deri...",She treated him with disdain.
6,Dregs,"the last drops of a liquid, mixed with little ...",No synonyms found for dregs,coffee dregs
7,Feint,(especially in sport) a movement that is inten...,ruse,N
8,Frenzy,a state of great activity and strong emotion t...,"burst, craze, excitement, fever, free-for-all,...",in a frenzy of activity/excitement/violence
9,Intimidate,to frighten or threaten somebody so that they ...,"alarm, awe, bully, coerce, constrain, daunt, d...",They were accused of intimidating people into ...


In [13]:
df.to_excel('/content/drive/MyDrive/Files/outputfile.xlsx', index=False)